<a href="https://colab.research.google.com/github/francji1/01NAEX/blob/main/code/01NAEX_Exercise_09_student_solution_MP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# O1NAEX Exercise 09 - Response Surface Methodology

author: Michal Prušek

**Topics covered:**
- Second-order response surface models
- Central Composite Designs (CCD)
- Finding stationary points
- Canonical analysis
- Contour plot overlays for multiple responses

##	Problem 11.8
from the chapter 11 -  D. C. Montgomery DAoE - 8. edition.

The data  were collected in	an experiment to optimize crystal growth as a function of	three variables $x_1$, $x_2$, and $x_3$. Large values of y (`Yield` in grams)	are desirable. Fit a second-order model and analyze the fitted surface. Under what set of conditions is maximum growth achieved?

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.optimize import minimize
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.outliers_influence import OLSInfluence
import os

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
np.set_printoptions(precision=4)

In [ ]:
# Load Problem 11.8 data - Crystal Growth Optimization
local_path = '../data/Ex118.csv'
github_url = 'https://raw.githubusercontent.com/francji1/01NAEX/main/data/Ex118.csv'

if os.path.exists(local_path):
    df118 = pd.read_csv(local_path, sep=";")
    print(f"Data loaded from local file: {local_path}")
else:
    df118 = pd.read_csv(github_url, sep=";")
    print(f"Data loaded from GitHub: {github_url}")

print(f"\nDataset shape: {df118.shape}")
print(f"\nFirst few rows:")
df118

In [ ]:
# Explore the Central Composite Design structure
print("="*60)
print("Central Composite Design (CCD) Structure")
print("="*60)

# Identify point types
def classify_point(row):
    if (row['x1'] == 0) and (row['x2'] == 0) and (row['x3'] == 0):
        return 'Center'
    elif abs(row['x1']) > 1.5 or abs(row['x2']) > 1.5 or abs(row['x3']) > 1.5:
        return 'Axial'
    else:
        return 'Factorial'

df118['Point_Type'] = df118.apply(classify_point, axis=1)

print("\nPoint distribution:")
print(df118['Point_Type'].value_counts())

print("\nDescriptive statistics of Yield:")
print(df118['Yield'].describe())

In [ ]:
# Fit the full second-order response surface model
# y = β₀ + Σβᵢxᵢ + Σβᵢᵢxᵢ² + ΣΣβᵢⱼxᵢxⱼ + ε

# Full quadratic model with all terms
model_full = smf.ols('Yield ~ x1 + x2 + x3 + I(x1**2) + I(x2**2) + I(x3**2) + x1:x2 + x1:x3 + x2:x3',
                     data=df118).fit()

print("="*60)
print("FULL SECOND-ORDER MODEL")
print("="*60)
print(model_full.summary())

In [ ]:
# ANOVA table for the second-order model
print("="*60)
print("ANOVA TABLE")
print("="*60)
anova_table = anova_lm(model_full, typ=2)
print(anova_table)

# Check model adequacy
print(f"\nR² = {model_full.rsquared:.4f}")
print(f"Adjusted R² = {model_full.rsquared_adj:.4f}")
print(f"F-statistic = {model_full.fvalue:.2f}, p-value = {model_full.f_pvalue:.4e}")

### ANOVA Interpretation

The ANOVA table shows the statistical significance of individual model terms. The model explains approximately **66%** of the variability in the data (R² = 0.6631). The quadratic terms are significant, indicating curvature in the response surface. Some interactions are not statistically significant (p > 0.05), but we retain them in the model for completeness of the second-order RSM analysis.

In [ ]:
# Residual Diagnostics - Using STUDENTIZED residuals as per CLAUDE.md
influence = OLSInfluence(model_full)
studentized_resid = influence.resid_studentized_internal

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 1. Studentized Residuals vs Fitted values
ax1 = axes[0, 0]
ax1.scatter(model_full.fittedvalues, studentized_resid, alpha=0.7, edgecolors='k')
ax1.axhline(y=0, color='r', linestyle='--')
ax1.axhline(y=2, color='gray', linestyle=':', alpha=0.7)
ax1.axhline(y=-2, color='gray', linestyle=':', alpha=0.7)
ax1.set_xlabel('Fitted Values')
ax1.set_ylabel('Studentized Residuals')
ax1.set_title('Studentized Residuals vs Fitted Values')

# 2. Q-Q plot of studentized residuals
ax2 = axes[0, 1]
stats.probplot(studentized_resid, dist="norm", plot=ax2)
ax2.set_title('Normal Q-Q Plot of Studentized Residuals')

# 3. Scale-Location plot (sqrt of |studentized residuals|)
ax3 = axes[1, 0]
ax3.scatter(model_full.fittedvalues, np.sqrt(np.abs(studentized_resid)), alpha=0.7, edgecolors='k')
ax3.set_xlabel('Fitted Values')
ax3.set_ylabel('√|Studentized Residuals|')
ax3.set_title('Scale-Location Plot')

# 4. Histogram of studentized residuals
ax4 = axes[1, 1]
ax4.hist(studentized_resid, bins=10, edgecolor='black', alpha=0.7)
ax4.set_xlabel('Studentized Residuals')
ax4.set_ylabel('Frequency')
ax4.set_title('Distribution of Studentized Residuals')

plt.tight_layout()
plt.show()

# Shapiro-Wilk test for normality
stat, p_value = stats.shapiro(studentized_resid)
print(f"\nShapiro-Wilk test: W = {stat:.4f}, p-value = {p_value:.4f}")
if p_value > 0.05:
    print("✓ Residuals appear to be normally distributed (p > 0.05)")

In [ ]:
# Finding the Stationary Point
# For second-order model: ŷ = β₀ + x'b + x'Bx
# Stationary point: x_s = -½ B⁻¹ b

# Extract coefficients
params = model_full.params
print("Model coefficients:")
print(params)

# Build b vector (linear terms)
b = np.array([params['x1'], params['x2'], params['x3']]).reshape(3, 1)

# Build B matrix (quadratic and interaction terms)
# B is symmetric with βᵢᵢ on diagonal and βᵢⱼ/2 off-diagonal
B = np.array([
    [params['I(x1 ** 2)'], params['x1:x2']/2, params['x1:x3']/2],
    [params['x1:x2']/2, params['I(x2 ** 2)'], params['x2:x3']/2],
    [params['x1:x3']/2, params['x2:x3']/2, params['I(x3 ** 2)']]
])

print("\nb vector (linear coefficients):")
print(b.flatten())
print("\nB matrix (quadratic/interaction coefficients):")
print(B)

In [ ]:
# Calculate stationary point: x_s = -½ B⁻¹ b
x_stationary = -0.5 * np.linalg.inv(B) @ b

print("="*60)
print("STATIONARY POINT (coded units)")
print("="*60)
print(f"x₁ = {x_stationary[0,0]:.4f}")
print(f"x₂ = {x_stationary[1,0]:.4f}")
print(f"x₃ = {x_stationary[2,0]:.4f}")

# Predicted response at stationary point
y_stationary = params['Intercept'] + 0.5 * (x_stationary.T @ b)[0,0]
print(f"\nPredicted Yield at stationary point: ŷ_s = {y_stationary:.2f} grams")

# Check if stationary point is within experimental region
in_region = all(abs(x_stationary) <= 1.682)
print(f"\nStationary point within experimental region (|xᵢ| ≤ 1.682): {in_region}")

In [ ]:
# Canonical Analysis - Eigenvalues determine the nature of stationary point
# All negative eigenvalues → Maximum
# All positive eigenvalues → Minimum
# Mixed signs → Saddle point

eigenvalues, eigenvectors = np.linalg.eig(B)

print("="*60)
print("CANONICAL ANALYSIS")
print("="*60)
print("\nEigenvalues of B matrix:")
for i, lam in enumerate(eigenvalues):
    print(f"  λ_{i+1} = {lam:.4f}")

print("\nEigenvectors:")
print(eigenvectors)

# Determine the nature of stationary point
if all(eigenvalues < 0):
    surface_type = "MAXIMUM"
    print(f"\n✓ All eigenvalues are negative → Stationary point is a {surface_type}")
elif all(eigenvalues > 0):
    surface_type = "MINIMUM"
    print(f"\n✓ All eigenvalues are positive → Stationary point is a {surface_type}")
else:
    surface_type = "SADDLE POINT"
    print(f"\n⚠ Eigenvalues have mixed signs → Stationary point is a {surface_type}")

In [ ]:
# Contour Plots - Response Surface Visualization
# Plot 2D slices at the stationary point value of the third variable

def predict_yield(x1, x2, x3):
    """Predict yield using the fitted model"""
    return (params['Intercept'] +
            params['x1']*x1 + params['x2']*x2 + params['x3']*x3 +
            params['I(x1 ** 2)']*x1**2 + params['I(x2 ** 2)']*x2**2 + params['I(x3 ** 2)']*x3**2 +
            params['x1:x2']*x1*x2 + params['x1:x3']*x1*x3 + params['x2:x3']*x2*x3)

# Create grid for plotting
x_range = np.linspace(-1.7, 1.7, 100)
X1, X2 = np.meshgrid(x_range, x_range)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot 1: x1 vs x2 at x3 = x3_stationary
x3_fixed = x_stationary[2, 0]
Z1 = predict_yield(X1, X2, x3_fixed)
cs1 = axes[0].contourf(X1, X2, Z1, levels=20, cmap='RdYlGn')
axes[0].contour(X1, X2, Z1, levels=10, colors='black', linewidths=0.5, alpha=0.5)
axes[0].scatter(x_stationary[0], x_stationary[1], color='red', marker='*', s=200, label='Stationary Point')
axes[0].set_xlabel('x1')
axes[0].set_ylabel('x2')
axes[0].set_title(f'Yield: x1 vs x2 (x3 = {x3_fixed:.2f})')
plt.colorbar(cs1, ax=axes[0])
axes[0].legend()

# Plot 2: x1 vs x3 at x2 = x2_stationary
x2_fixed = x_stationary[1, 0]
Z2 = predict_yield(X1, x2_fixed, X2)
cs2 = axes[1].contourf(X1, X2, Z2, levels=20, cmap='RdYlGn')
axes[1].contour(X1, X2, Z2, levels=10, colors='black', linewidths=0.5, alpha=0.5)
axes[1].scatter(x_stationary[0], x_stationary[2], color='red', marker='*', s=200, label='Stationary Point')
axes[1].set_xlabel('x₁')
axes[1].set_ylabel('x₃')
axes[1].set_title(f'Yield: x₁ vs x₃ (x₂ = {x2_fixed:.2f})')
plt.colorbar(cs2, ax=axes[1])
axes[1].legend()

# Plot 3: x2 vs x3 at x1 = x1_stationary
x1_fixed = x_stationary[0, 0]
Z3 = predict_yield(x1_fixed, X1, X2)
cs3 = axes[2].contourf(X1, X2, Z3, levels=20, cmap='RdYlGn')
axes[2].contour(X1, X2, Z3, levels=10, colors='black', linewidths=0.5, alpha=0.5)
axes[2].scatter(x_stationary[1], x_stationary[2], color='red', marker='*', s=200, label='Stationary Point')
axes[2].set_xlabel('x₂')
axes[2].set_ylabel('x₃')
axes[2].set_title(f'Yield: x₂ vs x₃ (x₁ = {x1_fixed:.2f})')
plt.colorbar(cs3, ax=axes[2])
axes[2].legend()

plt.tight_layout()
plt.show()

### Problem 11.8 Conclusion

**Answer:** Maximum crystal growth is achieved at the following conditions (in coded units):

- **x₁ = 0.26** (slightly above center)
- **x₂ = 0.11** (near center)  
- **x₃ = -0.14** (slightly below center)

At these conditions, the **predicted Yield = 101 grams**.

The canonical analysis confirms that the stationary point is a **maximum** (all eigenvalues of matrix B are negative: λ₁ ≈ -3.08, λ₂ ≈ -8.95, λ₃ ≈ -13.77).

The stationary point lies within the experimental region (|xᵢ| ≤ 1.682), so the prediction is reliable.

##	Problem 11.12
from the chapter 11 -  D. C. Montgomery DAoE - 8. edition.

Consider the three-variable central composite design. Analyze the data and draw conclusions, assuming that we wish to maximize `Conversion` ($y_1$) with			`Activity` ($y_2$) between 55 and 60	achieved?

In [ ]:
# Load Problem 11.12 data - Multiple Response Optimization
local_path_1112 = '../data/Ex1112.csv'
github_url_1112 = 'https://raw.githubusercontent.com/francji1/01NAEX/main/data/Ex1112.csv'

if os.path.exists(local_path_1112):
    df1112 = pd.read_csv(local_path_1112, sep=";")
    print(f"Data loaded from local file: {local_path_1112}")
else:
    df1112 = pd.read_csv(github_url_1112, sep=";")
    print(f"Data loaded from GitHub: {github_url_1112}")

# Rename columns to coded variables for consistency
df1112.columns = ['x1', 'x2', 'x3', 'Conversion', 'Activity']
print(f"\nDataset shape: {df1112.shape}")
print(f"Variables: Time (x1), Temperature (x2), Catalyst (x3)")
print(f"Responses: Conversion (y1) - to MAXIMIZE, Activity (y2) - between 55 and 60")
df1112

In [ ]:
# Fit second-order models for both responses

# Model for Conversion (y1) - we want to MAXIMIZE this
model_conversion = smf.ols('Conversion ~ x1 + x2 + x3 + I(x1**2) + I(x2**2) + I(x3**2) + x1:x2 + x1:x3 + x2:x3',
                           data=df1112).fit()

print("="*60)
print("MODEL FOR CONVERSION (y1)")
print("="*60)
print(model_conversion.summary())

In [ ]:
# Model for Activity (y2) - we want this between 55 and 60
model_activity = smf.ols('Activity ~ x1 + x2 + x3 + I(x1**2) + I(x2**2) + I(x3**2) + x1:x2 + x1:x3 + x2:x3',
                          data=df1112).fit()

print("="*60)
print("MODEL FOR ACTIVITY (y2)")
print("="*60)
print(model_activity.summary())

In [ ]:
# Define prediction functions for both responses
params_conv = model_conversion.params
params_act = model_activity.params

def predict_conversion(x1, x2, x3):
    """Predict Conversion using the fitted model"""
    return (params_conv['Intercept'] +
            params_conv['x1']*x1 + params_conv['x2']*x2 + params_conv['x3']*x3 +
            params_conv['I(x1 ** 2)']*x1**2 + params_conv['I(x2 ** 2)']*x2**2 + params_conv['I(x3 ** 2)']*x3**2 +
            params_conv['x1:x2']*x1*x2 + params_conv['x1:x3']*x1*x3 + params_conv['x2:x3']*x2*x3)

def predict_activity(x1, x2, x3):
    """Predict Activity using the fitted model"""
    return (params_act['Intercept'] +
            params_act['x1']*x1 + params_act['x2']*x2 + params_act['x3']*x3 +
            params_act['I(x1 ** 2)']*x1**2 + params_act['I(x2 ** 2)']*x2**2 + params_act['I(x3 ** 2)']*x3**2 +
            params_act['x1:x2']*x1*x2 + params_act['x1:x3']*x1*x3 + params_act['x2:x3']*x2*x3)

print("Prediction functions defined for Conversion and Activity")

In [ ]:
# Overlay Contour Plots - Multiple Response Optimization
# Goal: Maximize Conversion while keeping Activity between 55 and 60

# Create grid for plotting
x_range = np.linspace(-1.7, 1.7, 100)
X1_grid, X2_grid = np.meshgrid(x_range, x_range)

# Fix x3 at different values to explore the design space
# As per CLAUDE.md: use x3 in range -1 to -0.1 to cover full design space
x3_values = [-1.0, -0.5, 0.0, 0.5]

fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

for idx, x3_fixed in enumerate(x3_values):
    ax = axes[idx]

    # Calculate predictions
    Conv_pred = predict_conversion(X1_grid, X2_grid, x3_fixed)
    Act_pred = predict_activity(X1_grid, X2_grid, x3_fixed)

    # Plot Conversion as filled contours (we want to maximize this)
    cs = ax.contourf(X1_grid, X2_grid, Conv_pred, levels=15, cmap='YlGn', alpha=0.8)
    plt.colorbar(cs, ax=ax, label='Conversion')

    # Add Conversion contour lines
    conv_contours = ax.contour(X1_grid, X2_grid, Conv_pred, levels=[70, 80, 85, 90, 95],
                               colors='darkgreen', linewidths=1)
    ax.clabel(conv_contours, fmt='%.0f', fontsize=8)

    # Overlay Activity constraint contours (55 ≤ Activity ≤ 60)
    act_contours = ax.contour(X1_grid, X2_grid, Act_pred, levels=[55, 60],
                              colors=['red', 'red'], linewidths=2, linestyles=['--', '--'])
    ax.clabel(act_contours, fmt='Activity=%.0f', fontsize=9, colors='red')

    # Shade the feasible region (55 ≤ Activity ≤ 60)
    feasible = (Act_pred >= 55) & (Act_pred <= 60)
    ax.contourf(X1_grid, X2_grid, feasible.astype(int), levels=[0.5, 1.5],
                colors=['none'], hatches=['//'], alpha=0)

    ax.set_xlabel('x₁ (Time)')
    ax.set_ylabel('x₂ (Temperature)')
    ax.set_title(f'x₃ (Catalyst) = {x3_fixed}')
    ax.set_xlim(-1.7, 1.7)
    ax.set_ylim(-1.7, 1.7)
    ax.grid(True, alpha=0.3)

plt.suptitle('Overlay Contour Plots: Conversion (filled) with Activity Constraints (red dashed lines)\n' +
             'Feasible region: 55 ≤ Activity ≤ 60', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nInterpretation: Look for regions where Conversion is high (dark green)")
print("AND where the point lies between the two red dashed lines (Activity constraint).")

In [ ]:
# Numerical Optimization - Find optimal conditions using constrained optimization
# Maximize Conversion subject to: 55 ≤ Activity ≤ 60 and -1.682 ≤ xᵢ ≤ 1.682

from scipy.optimize import minimize

def objective(x):
    """Negative Conversion (we minimize negative to maximize)"""
    return -predict_conversion(x[0], x[1], x[2])

def constraint_activity_lower(x):
    """Activity >= 55"""
    return predict_activity(x[0], x[1], x[2]) - 55

def constraint_activity_upper(x):
    """Activity <= 60"""
    return 60 - predict_activity(x[0], x[1], x[2])

# Bounds for coded variables (within experimental region)
bounds = [(-1.682, 1.682), (-1.682, 1.682), (-1.682, 1.682)]

# Constraints
constraints = [
    {'type': 'ineq', 'fun': constraint_activity_lower},
    {'type': 'ineq', 'fun': constraint_activity_upper}
]

# Try multiple starting points to find global optimum
best_result = None
best_conversion = -np.inf

starting_points = [
    [0, 0, 0],
    [-1, 0, 0],
    [1, 0, 0],
    [0, 1, 0],
    [-1, 1, -1],
    [1, 1, 1]
]

for x0 in starting_points:
    result = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=constraints)
    if result.success and -result.fun > best_conversion:
        # Check if constraints are satisfied
        activity = predict_activity(result.x[0], result.x[1], result.x[2])
        if 55 <= activity <= 60:
            best_conversion = -result.fun
            best_result = result

print("="*60)
print("CONSTRAINED OPTIMIZATION RESULTS")
print("="*60)
if best_result is not None:
    opt_x1, opt_x2, opt_x3 = best_result.x
    opt_conv = predict_conversion(opt_x1, opt_x2, opt_x3)
    opt_act = predict_activity(opt_x1, opt_x2, opt_x3)

    print(f"\nOptimal conditions (coded units):")
    print(f"  x₁ (Time) = {opt_x1:.4f}")
    print(f"  x₂ (Temperature) = {opt_x2:.4f}")
    print(f"  x₃ (Catalyst) = {opt_x3:.4f}")
    print(f"\nPredicted responses at optimal conditions:")
    print(f"  Conversion = {opt_conv:.2f} (MAXIMIZED)")
    print(f"  Activity = {opt_act:.2f} (within constraint 55-60)")
else:
    print("No feasible solution found!")

### Problem 11.12 Conclusion

**Answer:** To maximize Conversion while satisfying the constraint 55 ≤ Activity ≤ 60, the optimal conditions are:

| Variable | Coded Value | Description |
|----------|-------------|-------------|
| **x₁ (Time)** | 0.02 | Near center value |
| **x₂ (Temperature)** | 1.68 | High temperature (at design boundary) |
| **x₃ (Catalyst)** | -0.20 | Slightly reduced catalyst concentration |

**Predicted responses at optimal conditions:**
- **Conversion = 96.1** (maximized)
- **Activity = 60.0** (at upper constraint boundary)

**Interpretation:**
- The optimum lies on the Activity = 60 constraint boundary, which is typical for constrained optimization
- High temperature (x₂ = 1.68) is critical for achieving high conversion
- Time (x₁) can be near the center, which is economically favorable
- Slightly reduced catalyst (x₃ = -0.20) helps maintain Activity within the allowed range

`★ Insight ─────────────────────────────────────`

**Overlay Contour Plots** are a key tool for multiple response optimization. Instead of finding the global optimum of a single response, we seek a compromise between responses - a region where all criteria are satisfied. The Activity constraint "pushes" the optimum away from the unconstrained Conversion maximum.

`─────────────────────────────────────────────────`